#### Imports

In [9]:
import pandas as pd
import numpy as np

from src.scripts.utils import check_series
from src.scripts.market_api import get_fred_time_series, get_yfinance_time_series

#### Check S&P500 Total return data

In [10]:
sptr = get_yfinance_time_series('^SP500TR', '1985-01-01', '2000-08-01', '1mo')

sp500_cochrane = pd.read_csv('../../data/cochrane_data/s&p500.csv')
sp500_cochrane['date'] = sp500_cochrane.apply(lambda x: pd.to_datetime(f'{int(x.Year)}-{int(x.Month)}-01'), axis=1)
sp500_cochrane = sp500_cochrane.drop(columns=['Year', 'Month']).set_index('date')
sp500_cochrane = sp500_cochrane[sp500_cochrane.index > '1988-01-01']

logret_cochrane = np.log(sp500_cochrane.pct_change().dropna() + 1)
logret = np.log(sptr.pct_change().dropna() + 1)

check_series(logret, logret_cochrane)

#### Check t-Bills 3 Months data

In [11]:
tb3ms_cochrane = pd.read_csv('../../data/cochrane_data/tb3ms.csv')
tb3ms_cochrane['date'] = tb3ms_cochrane['date'].apply(lambda x: pd.to_datetime(f'{str(x)[:4]}-{str(x)[4:]}-01'))
tb3ms_cochrane = tb3ms_cochrane.set_index('date')

tb3ms = get_fred_time_series("TB3MS", '1987-01-01', '2000-08-01')
tb3ms = tb3ms.set_index(tb3ms_cochrane.index)

check_series(tb3ms, tb3ms_cochrane)

#### Retrieve new data

In [12]:
start_date = '2000-01-01'
end_date = '2022-04-30'
frequency ='1mo'

sptr_new = get_yfinance_time_series('^SP500TR', start_date, end_date, frequency)
nasdaq_new = get_yfinance_time_series('^IXIC', start_date, end_date, frequency)
tb3ms_new = get_fred_time_series('TB3MS', start_date, end_date)

df = pd.concat([tb3ms_new, sptr_new, nasdaq_new], axis=1)

In [13]:
df

,TB3MS,^SP500TR,^IXIC
date,,,
2000-01-01,5.32,1919.839966,3940.350098
2000-02-01,5.55,1883.500000,4696.689941
2000-03-01,5.69,2067.760010,4572.830078
2000-04-01,5.66,2005.550049,3860.659912
2000-05-01,5.79,1964.400024,3400.909912
...,...,...,...
2021-12-01,0.06,9986.700195,15644.969727
2022-01-01,0.15,9469.919922,14239.879883
2022-02-01,0.33,9186.370117,13751.400391
